In [1]:
from movie_recommender import MovieRecommender
import timeit
import pandas as pd

In [2]:
started = timeit.default_timer()
movies_file = 'data/ml-10M100K/movies.dat'
ratings_file = 'data/ml-10M100K/ratings.dat'
movie_rec = MovieRecommender(movies_filename=movies_file, ratings_filename=ratings_file)
elapsed = timeit.default_timer() - started
print(f'Read dataframes in {elapsed}ms')

print(movie_rec.ratings_df['user_id'].nunique())

found pickle at c:\Users\john.french\movie_recs\data\pickled\movie_ratings.pkl
Read dataframes in 23.191476400010288ms
69878


In [3]:
# replace pipe character with space
movie_rec.movies_df['genres'] = movie_rec.movies_df['genres'].str.replace('|', ' ')
rating_counts = movie_rec.ratings_df.groupby('movie_id').agg('rating').count()
rating_counts.name = 'num_ratings'
rating_avg = movie_rec.ratings_df.groupby('movie_id').agg('rating').mean()
rating_avg.name = 'avg_rating'
most_rated = rating_counts.sort_values(ascending=False).head(1000)
most_rated = pd.merge(most_rated, movie_rec.movies_df, on="movie_id")
most_rated = pd.merge(rating_avg, most_rated, on="movie_id")
most_rated
most_rated.sort_values(by='num_ratings', ascending=False, inplace=True)

C:\Users\john.french\AppData\Local\Temp\ipykernel_576\2230656454.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  movie_rec.movies_df['genres'] = movie_rec.movies_df['genres'].str.replace('|', ' ')


In [ ]:
ctr = 0
limit = 15
for idx, row in most_rated.head(10).iterrows():
    print(row['name'], idx)
    ctr += 1
    if ctr > limit:
        break

In [ ]:
descriptions_df = pd.read_json('data/metadata/movie_descriptions.json')
most_rated = pd.merge(descriptions_df, most_rated, on='movie_id', how='right')
most_rated = most_rated.head(1000)
most_rated.set_index('movie_id', inplace=True)
most_rated['movie_id'] = most_rated.index
most_rated.columns
most_rated.to_json('data/metadata/top_rated_1000.json', orient='records')

In [ ]:
import os
import openai

# try to load the api key from file
secrets_filename = os.path.join(os.path.abspath(
    os.path.dirname('eda.ipynb')), 'ai_posters/secret2.txt')
try:
    with open(secrets_filename, 'r') as f:
        api_key = f.read().strip()
        openai.api_key = api_key
except Exception as e:
    print(e)

In [ ]:
import json
import openai
# fill in the descriptions with chatgpt
# Note: you need to be using OpenAI Python v0.27.0 for the code below to work

ctr = 0
limit = 500
# most_rated.reset_index(inplace=True)
for i in most_rated['movie_id']:
  if ctr >= limit:
    break
  ctr += 1
  try:
    movie_json_str = most_rated[most_rated.movie_id == i].to_json(orient='records')
    print(movie_json_str)
    prompt = """"
    for the movie and movie_id, describe the plot in 50 words or less without spoilers. 
    Swap all names used for similar ones including the title of the movie. do not mention the rating metadata,
    only focus on the movie's plot and characters. Make sure your ouput is valid json with opening and closing brackets
    ###json begins
    """
    prompt += movie_json_str
    prompt += '\n###json ends'

    res = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "system", "content": """You are a helpful assistant that
                  transforms movie titles and descriptions. Output is json with 
                  two fields: substituted_name, and description. Ensure that 
                  the fields are correctly formatted as .json. Only return one object."""},
            {"role": "user", "content": prompt}
        ]
    )
    choices = res['choices']
    # print(choices)
    if len(choices) != 1:
      print('1 choice was not returned')
    for choice in choices:
      print(choice['message']['content'])
      desc = json.loads(str(choice['message']['content']))
      if type(desc) ==type(list()):
        desc = desc[0]
      print(desc)
      sub_name = desc['substituted_name']
      description = desc['description']
      print('sub name of ', sub_name)
      most_rated.loc[most_rated.movie_id == i, 'substituted_name'] = sub_name
      most_rated.loc[most_rated.movie_id == i, 'description'] = description
      print(most_rated.loc[i])
  except Exception as e:
    print(f'Exception with req for {movie_json_str}', e)
    continue
most_rated.to_json('data/metadata/top_rated_1000_new_strings.json', orient='records')

In [4]:
from ai_posters import test_api_route

ctr = 0
offset = 532
limit = 600
for idx, row in most_rated.iterrows():
    movie_title = row['name']
    genres = row['genres']
    movie_id = idx
    ctr += 1
    if ctr >= offset:
        test_api_route.make_poster_for_movie(movie_title, genres, str(movie_id))
    if ctr == limit:
        break

<Response [200]>
208242082
Images saved successfully
<Response [200]>
736485956
Images saved successfully
<Response [200]>
3385728743
Images saved successfully
<Response [200]>
2612964663
Images saved successfully
<Response [200]>
2211936589
Images saved successfully
<Response [200]>
707613815
Images saved successfully
<Response [200]>
2603439747
Images saved successfully
<Response [200]>
3644911459
Images saved successfully
<Response [200]>
2799435855
Images saved successfully
<Response [200]>
2887982880
Images saved successfully
<Response [200]>
142519003
Images saved successfully
<Response [200]>
1781188645
Images saved successfully
<Response [200]>
593292835
Images saved successfully
<Response [200]>
902294266
Images saved successfully
<Response [200]>
1143565822
Images saved successfully
<Response [200]>
2037868935
Images saved successfully
<Response [200]>
1913893066
Images saved successfully
<Response [200]>
820184215
Images saved successfully
<Response [200]>
2125080271
Images 

In [ ]:
# clean_titles = [movie_rec.get_movie_title(m) for m in set(movie_rec.ratings_df.index.values)]
import pandas as pd

# create a DataFrame containing the movie titles and ids
movies_df = pd.DataFrame({'title': movie_rec.movies_df['name']})

# count the occurrences of each title using the value_counts() method
title_counts = movies_df['title'].value_counts()

# filter out the titles that appear only once using boolean indexing
popular_titles = title_counts[title_counts > 1].index.tolist()

# get the corresponding movie ids for the popular titles
popular_movie_ids = movies_df.index.get_level_values('movie_id')[movies_df['title'].isin(popular_titles)].unique().tolist()

print(popular_movie_ids)

In [ ]:
u = int(movie_rec.mrm_df.shape[0])
# u = 2
# print(type(u), u)

# movie_rec.new_add_movie_rating(movie_id=1219, user_id=u, rating=5) # Psycho
# How to make an american quilt
# movie_rec.new_add_movie_rating(movie_id=46, user_id=u, rating=5) 
# movie_rec.new_add_movie_rating(movie_id=16, user_id=u, rating=5) # Casino
# movie_rec.new_add_movie_rating(movie_id=2596, user_id=u, rating=5) # Rocky Horror
# movie_rec.new_add_movie_rating(movie_id=2571, user_id=u, rating=5) # The Matrix
movie_rec.new_add_movie_rating(movie_id=107, user_id=u, rating=5) # Muppet Treasure Island
movie_rec.new_add_movie_rating(movie_id=5080, user_id=u, rating=5) # Slackers
# movie_rec.new_add_movie_rating(movie_id=30, user_id=u, rating=5)
# movie_rec.new_add_movie_rating(movie_id=232, user_id=u, rating=5)
# movie_rec.new_add_movie_rating(movie_id=858, user_id=u, rating=5)
# movie_rec.new_add_movie_rating(movie_id=48, user_id=u, rating=5)
# movie_rec.new_add_movie_rating(movie_id=1213, user_id=u, rating=5)
# movie_rec.new_add_movie_rating(movie_id=858, user_id=u, rating=5)

movie_rec.refit_model()


In [ ]:
movie_rec.get_user_based_recommendation(u)

In [ ]:
# movie_rec.get_user_movies(69877)
for m in movie_rec.get_user_movies(69877):
    print(movie_rec.get_movie_title(m), movie_rec.get_user_movie_rating(69877, m))
# movie_rec.get_user_movie_rating(8546, 1)
# movie_rec.get_user_movie_rating(8546, 2)
# print(f'User {u} has seen...')
# for m in movie_rec.get_user_movies(u):
#     print(movie_rec.get_movie_title(m))

In [ ]:
users, dists = movie_rec.get_similar_users(u)
# for rec_user in users:
#     # print(f'{rec_user}\'s movies')
#     for m in movie_rec.get_user_movies(rec_user):
#         # print(movie_rec.get_movie_title(m))

# print(movie_rec.get_movie_title(20))
# print(movie_rec.get_movie_title(25))
# print(movie_rec.get_movie_title(50))

In [ ]:
user_id = movie_rec.get_next_avail_user_id()
# movie_rec.add_movie_rating(movie_id=1, user_id= user_id, rating= 5) # Toy Story
# movie_rec.add_movie_rating(movie_id=2, user_id= user_id, rating= 5) # Jumanji
# movie_rec.add_movie_rating(movie_id=48, user_id= user_id, rating= 5) # Pocahontas
# movie_rec.add_movie_rating(movie_id=20, user_id= user_id, rating= 5) # Money Train
# movie_rec.add_movie_rating(movie_id=25, user_id= user_id, rating= 5) # Leaving Las Vegas
# movie_rec.add_movie_rating(movie_id=50, user_id= user_id, rating= 5) # The Usual Suspects
movie_rec.ratings_df.head()

In [ ]:
import numpy as np


# return the difference between average ratings for two movies
def get_movie_mean_rating_error(movie1_id: int, movie2_id: int):
    movie1_ratings = np.mean(
        movie_rec.ratings_df.loc[movie_rec.ratings_df.index == movie1_id]['rating'].values)
    movie2_ratings = np.mean(
        movie_rec.ratings_df.loc[movie_rec.ratings_df.index == movie2_id]['rating'].values)
    return abs(movie1_ratings - movie2_ratings)


# get_movie_mean_rating_error(599, 700)
# movie_rec.get_movie_users(5)
print(movie_rec.get_movie_title(1))
print(movie_rec.get_movie_title(2))
print(len(movie_rec.get_movie_users(1)))
print(len(movie_rec.get_movie_users(2)))
print(len(movie_rec.get_common_users(1, 2)))

In [ ]:
def get_movie_rating_pearson_coeff(movie1_id: int, movie2_id: int)-> float:
    common_users = movie_rec.get_common_users(movie1_id=movie1_id, movie2_id=movie2_id)
    movie1_ratings = [movie_rec.get_user_movie_rating(user_id=u, movie_id=movie1_id) for u in common_users]
    movie2_ratings = [movie_rec.get_user_movie_rating(user_id=u, movie_id=movie2_id) for u in common_users]
    coeff = np.corrcoef(np.array(movie1_ratings), np.array(movie2_ratings))[0, 1]
    movie1_title = movie_rec.get_movie_title(movie1_id)
    movie2_title = movie_rec.get_movie_title(movie2_id)
    print(f'{movie1_title} and {movie2_title} have a correlation of {"{:.2f}".format(coeff)} from {len(common_users)} users who have rated both')

get_movie_rating_pearson_coeff(1, 3)